In [4]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce RTX 3080 Ti


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

# Custom Dataset 생성
class FakePPGDataset(Dataset):
    def __init__(self, num_samples=1000, signal_length=250, num_classes=2, sampling_rate=34):
        np.random.seed(42)
        self.num_samples = num_samples
        self.signal_length = signal_length
        self.num_classes = num_classes
        self.sampling_rate = sampling_rate

        # 가짜 PPG 데이터 생성 (정규 분포)
        self.data = np.random.normal(0, 1, (num_samples, signal_length))
        
        # 라벨 생성 (0 또는 1)
        self.labels = np.random.randint(0, num_classes, num_samples)

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        signal = self.data[idx]
        label = self.labels[idx]
        # PyTorch 텐서로 변환
        return torch.tensor(signal, dtype=torch.float32).unsqueeze(0), torch.tensor(label, dtype=torch.long)

# 데이터셋 준비
train_dataset = FakePPGDataset(num_samples=800)
test_dataset = FakePPGDataset(num_samples=200)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleSleepPPGModel(nn.Module):
    def __init__(self, num_classes=2):  # num_classes는 분류할 클래스 수
        super(SimpleSleepPPGModel, self).__init__()
        
        # Convolutional Layers
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # Batch Normalization
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(32)
        self.bn3 = nn.BatchNorm1d(64)
        
        # MaxPooling Layers
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(64 * 62, 128)  # 조정된 입력 크기 (64 feature maps, 길이 62로 감소)
        self.fc2 = nn.Linear(128, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # Convolutional Layers with ReLU, BatchNorm, and MaxPooling
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # Conv1 -> ReLU -> BatchNorm -> MaxPooling
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # Conv2 -> ReLU -> BatchNorm -> MaxPooling
        x = F.relu(self.bn3(self.conv3(x)))            # Conv3 -> ReLU -> BatchNorm (마지막 레이어는 Pooling 생략 가능)
        
        # Flattening
        x = torch.flatten(x, start_dim=1)
        
        # Fully Connected Layers
        x = F.relu(self.fc1(x))  # Fully Connected Layer + ReLU
        x = self.dropout(x)      # Dropout
        x = self.fc2(x)          # Output Layer (분류)
        
        return x


In [12]:
# 하이퍼파라미터
num_epochs = 10
learning_rate = 0.001

# 모델, 손실 함수, 옵티마이저 초기화
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleSleepPPGModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for signals, labels in train_loader:
        signals, labels = signals.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(signals)
        loss = criterion(outputs, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")


Epoch [1/10], Loss: 0.8067
Epoch [2/10], Loss: 0.6601
Epoch [3/10], Loss: 0.6136
Epoch [4/10], Loss: 0.5261
Epoch [5/10], Loss: 0.4366
Epoch [6/10], Loss: 0.3575
Epoch [7/10], Loss: 0.2594
Epoch [8/10], Loss: 0.2180
Epoch [9/10], Loss: 0.1173
Epoch [10/10], Loss: 0.0853
